In [26]:
import os
import pandas as pd

def load_bbc_dataset(base_path='bbc'):
    """
    Считывает все txt-файлы из подпапок папки base_path,
    формирует список (текст, метка) и возвращает pd.DataFrame.
    """
    data = []

    for label_folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, label_folder)
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                if filename.endswith('.txt'):
                    file_path = os.path.join(folder_path, filename)

                    # Пробуем сначала открыть как utf-8, при ошибке — как latin-1
                    try:
                        with open(file_path, 'r', encoding='utf-8') as f:
                            text = f.read()
                    except UnicodeDecodeError:
                        with open(file_path, 'r', encoding='latin-1') as f:
                            text = f.read()

                    data.append((text, label_folder))

    df = pd.DataFrame(data, columns=['text', 'label'])
    return df



In [27]:
base_path = "bbc"  # Путь к папке, где лежат подпапки business, entertainment и т.д.
df = load_bbc_dataset(base_path)
print(df.head())
print(df['label'].value_counts())

                                                text     label
0  Ad sales boost Time Warner profit\n\nQuarterly...  business
1  Dollar gains on Greenspan speech\n\nThe dollar...  business
2  Yukos unit buyer faces loan claim\n\nThe owner...  business
3  High fuel prices hit BA's profits\n\nBritish A...  business
4  Pernod takeover talk lifts Domecq\n\nShares in...  business
label
sport            511
business         506
politics         417
tech             401
entertainment    386
Name: count, dtype: int64


In [28]:
# 1. Разделение данных на Train (80%), Validation (10%) и Test (10%)
# Предполагается, что исходный DataFrame с колонками 'clean_text' (очищенный текст) и 'label' уже подготовлен (см. предыдущий шаг)
train_df, temp_df = train_test_split(df, test_size=0.20, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.50, random_state=42, stratify=temp_df['label'])

print("Размер обучающей выборки:", train_df.shape)
print("Размер валидационной выборки:", val_df.shape)
print("Размер тестовой выборки:", test_df.shape)

Размер обучающей выборки: (1776, 2)
Размер валидационной выборки: (222, 2)
Размер тестовой выборки: (223, 2)


In [29]:
import re
from bs4 import BeautifulSoup
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Функция очистки текста – стандартная предобработка (удаляет HTML, приводит к нижнему регистру, удаляет лишние пробелы, знаки препинания, стоп-слова и выполняет стемминг)
def clean_text(text):
    # 1. Удаление HTML-тегов
    text = BeautifulSoup(text, "html.parser").get_text()
    # 2. Замена множественных пробелов, переносов строк и табуляций на один пробел
    text = re.sub(r'\s+', ' ', text)
    # 3. Приведение к нижнему регистру
    text = text.lower()
    # 4. Удаление символов, отличных от букв (оставляем только латинские буквы и пробелы)
    text = re.sub(r'[^a-z\s]', '', text)
    # 5. Токенизация
    tokens = word_tokenize(text)
    # 6. Удаление стоп-слов
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    # 7. Стемминг
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    # Объединение токенов в строку
    return ' '.join(tokens)

# Класс, который объединяет всю цепочку: векторизацию, обучение классификатора и инференс.
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

class NewsClassifier:
    def __init__(self, ngram_range=(1, 2), min_df=5, C=1.0):
        # Инициализируем TF-IDF векторизатор и классификатор Logistic Regression
        self.vectorizer = TfidfVectorizer(ngram_range=ngram_range, min_df=min_df)
        self.clf = LogisticRegression(random_state=42, max_iter=200, C=C)
    
    def fit(self, texts, labels):
        """Обучение модели на списке сырой текстов и меток. При этом каждый текст проходит очистку."""
        # Применяем функцию очистки к каждому тексту
        cleaned_texts = [clean_text(text) for text in texts]
        # Векторизуем очищенные тексты
        X = self.vectorizer.fit_transform(cleaned_texts)
        # Обучаем классификатор
        self.clf.fit(X, labels)
    
    def predict(self, text):
        """
        Принимает на вход один сырой текст новости,
        выполняет очистку, векторизацию, и возвращает:
            - предсказанную метку,
            - словарь вероятностей по классам.
        """
        cleaned = clean_text(text)
        X = self.vectorizer.transform([cleaned])
        label = self.clf.predict(X)[0]
        probs = self.clf.predict_proba(X)[0]
        probabilities = dict(zip(self.clf.classes_, probs))
        return label, probabilities
    
    def predict_batch(self, texts):
        """
        Предсказывает для списка текстов.
        Возвращает:
            - массив предсказанных меток,
            - список словарей с вероятностями.
        """
        cleaned_texts = [clean_text(text) for text in texts]
        X = self.vectorizer.transform(cleaned_texts)
        labels = self.clf.predict(X)
        prob_array = self.clf.predict_proba(X)
        prob_list = [dict(zip(self.clf.classes_, probs)) for probs in prob_array]
        return labels, prob_list
    
    def evaluate(self, texts, true_labels):
        """Выводит отчет по метрикам (accuracy, classification report) на переданном наборе."""
        pred_labels, _ = self.predict_batch(texts)
        print(classification_report(true_labels, pred_labels))
        print("Accuracy:", accuracy_score(true_labels, pred_labels))

In [30]:
# Инициализация модели (параметры можно подбирать через GridSearchCV отдельно)
classifier = NewsClassifier(ngram_range=(1,2), min_df=5, C=1.0)
# Обучаем на обучающей выборке (используем сырой текст, а очистка происходит внутри fit)
classifier.fit(train_df['text'], train_df['label'])

# Оценка на валидационной выборке
print("Метрики на валидационной выборке:")
classifier.evaluate(val_df['text'], val_df['label'])

# Итоговая оценка на тестовой выборке (эта выборка не трогалась при обучении)
print("Метрики на тестовой выборке:")
classifier.evaluate(test_df['text'], test_df['label'])

Метрики на валидационной выборке:
               precision    recall  f1-score   support

     business       1.00      0.98      0.99        50
entertainment       1.00      1.00      1.00        39
     politics       0.98      0.98      0.98        42
        sport       1.00      1.00      1.00        51
         tech       0.98      1.00      0.99        40

     accuracy                           0.99       222
    macro avg       0.99      0.99      0.99       222
 weighted avg       0.99      0.99      0.99       222

Accuracy: 0.990990990990991
Метрики на тестовой выборке:
               precision    recall  f1-score   support

     business       1.00      1.00      1.00        51
entertainment       1.00      0.97      0.99        38
     politics       0.98      1.00      0.99        42
        sport       1.00      1.00      1.00        52
         tech       1.00      1.00      1.00        40

     accuracy                           1.00       223
    macro avg       1.00

In [31]:
# Пример инференса одного образца
sample_text = "The government announced new policies in tech innovation to boost the local economy."
pred_label, pred_probs = classifier.predict(sample_text)

print("\nПример инференса одного образца:")
print("Исходный текст:", sample_text)
print("Предсказанная метка:", pred_label)
print("Вероятности по классам:")
for lbl, prob in pred_probs.items():
    print(f"  {lbl}: {prob:.4f}")


Пример инференса одного образца:
Исходный текст: The government announced new policies in tech innovation to boost the local economy.
Предсказанная метка: business
Вероятности по классам:
  business: 0.4136
  entertainment: 0.1186
  politics: 0.2331
  sport: 0.1280
  tech: 0.1066


In [32]:
import os
import pandas as pd

def load_bbc_new_dataset(base_path='bbc_new'):
    """
    Считывает все txt-файлы из подпапок в base_path.
    В каждой подпапке имя соответствует метке.
    Возвращает DataFrame с колонками 'text' и 'true_label'.
    """
    data = []
    # Проходим по каждой подпапке
    for label_folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, label_folder)
        if os.path.isdir(folder_path):
            # Проходим по всем .txt файлам в подпапке
            for filename in os.listdir(folder_path):
                if filename.endswith('.txt'):
                    file_path = os.path.join(folder_path, filename)
                    try:
                        with open(file_path, 'r', encoding='utf-8') as f:
                            text = f.read()
                    except Exception as e:
                        print(f"Ошибка чтения файла {file_path}: {e}")
                        continue
                    data.append((text, label_folder))
    # Формируем DataFrame
    df_new = pd.DataFrame(data, columns=['text', 'true_label'])
    return df_new

# Загружаем новый датасет из папки 'bbc_new'
df_new = load_bbc_new_dataset("bbc_new")
print("Первичные данные нового датасета:")
print(df_new.head())

# Предполагается, что вы обучили модель и сохранили объект classifier
# Например: 
# classifier = NewsClassifier(ngram_range=(1,2), min_df=5, C=1.0)
# classifier.fit(train_texts, train_labels)
# Теперь мы будем использовать метод predict_batch для применения модели к набору новостей.

# Получаем предсказания для всех текстов нового датасета
pred_labels, pred_prob_list = classifier.predict_batch(df_new['text'])

# Добавляем столбец с предсказаниями в DataFrame
df_new['pred_label'] = pred_labels
df_new['pred_prob'] = pred_prob_list

# Выводим первые несколько строк с результатами
print("\nРезультаты инференса на новом датасете:")
print(df_new[['true_label', 'pred_label', 'pred_prob']].head(10))

# Если для нового набора новостей известны истинные метки (как столбец 'true_label'),
# можно вычислить метрики (например, classification_report)
from sklearn.metrics import classification_report
print("\nОтчет по предсказаниям на новом датасете:")
print(classification_report(df_new['true_label'], df_new['pred_label']))


Первичные данные нового датасета:
                                                text true_label
0  Investors facing tariff turmoil: 'It's fastest...   business
1  Prada to buy rival fashion brand Versace for $...   business
2  How exposed is the UK to Trump's tariff chaos?...   business
3  UK economy grew more than expected in February...   business
4  Watch: Why US markets skyrocketed after Trump ...   business

Результаты инференса на новом датасете:
      true_label     pred_label  \
0       business       business   
1       business       business   
2       business       business   
3       business       business   
4       business       business   
5  entertainment  entertainment   
6  entertainment  entertainment   
7  entertainment  entertainment   
8  entertainment  entertainment   
9  entertainment  entertainment   

                                           pred_prob  
0  {'business': 0.7730855920339225, 'entertainmen...  
1  {'business': 0.6826105821819952, 'entertai

In [9]:
# !pip install transformers datasets accelerate -q

import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# ==== 1) Подготовка текстов и лейблов ====
# У нас есть:
#   train_df, val_df, test_df
# со столбцами 'text' (или 'clean_text') и 'label'.
# Для BERT лучше взять 'text' с минимальной предобработкой.

label_list = df['label'].unique().tolist()   # ['business', 'entertainment', 'politics', 'sport', 'tech']
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

def encode_labels(label_str):
    return label_to_id[label_str]

train_df['label_id'] = train_df['label'].apply(encode_labels)
val_df['label_id']   = val_df['label'].apply(encode_labels)
test_df['label_id']  = test_df['label'].apply(encode_labels)

# Превращаем pandas.DataFrame в huggingface Dataset
train_ds = Dataset.from_pandas(train_df[['text', 'label_id']])
val_ds   = Dataset.from_pandas(val_df[['text', 'label_id']])
test_ds  = Dataset.from_pandas(test_df[['text', 'label_id']])

train_ds = train_ds.rename_column("label_id", "labels")
val_ds = val_ds.rename_column("label_id", "labels")
test_ds = test_ds.rename_column("label_id", "labels")


dataset = DatasetDict({
    'train': train_ds,
    'validation': val_ds,
    'test': test_ds
})

# ==== 2) Токенизация ====
# Инициализируем DistilBert-токенизатор
model_checkpoint = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_checkpoint)

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",      # или "longest", в зависимости от задачи
        truncation=True,
        max_length=256             # подберите по своим ресурсам и данным
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# ==== 3) Создание модели DistilBERT ====
# Число классов = 5 (business, entertainment, politics, sport, tech)
num_labels = 5

model = DistilBertForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    id2label=id_to_label,
    label2id=label_to_id
)

# ==== 4) Метрики ====
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ==== 5) Настройки обучения ====
batch_size = 8
logging_steps = len(tokenized_dataset['train']) // batch_size

training_args = TrainingArguments(
    output_dir="distilbert-bbc",       
    eval_strategy="epoch",       # или "steps"
    save_strategy="epoch",
    logging_steps=logging_steps,
    num_train_epochs=3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=5e-5,
    load_best_model_at_end=True,       # чтобы иметь лучшую модель в конце
    metric_for_best_model="accuracy",  # какую метрику использовать при сохранении best
)

# ==== 6) Trainer ====
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# ==== 7) Запуск обучения (fine-tuning DistilBERT) ====
trainer.train()

# ==== 8) Оценка на тестовой выборке ====
test_results = trainer.evaluate(tokenized_dataset["test"])
print(test_results)
# {'eval_loss': ..., 'eval_accuracy': ..., 'eval_precision': ..., 'eval_recall': ..., 'eval_f1': ...}

# ==== 9) Пример инференса одного текста ====
sample_text = "A new movie starring popular actors is releasing soon."
# Токенизируем
encoded_input = tokenizer([sample_text], padding=True, truncation=True, max_length=256, return_tensors="pt")
# Прогоняем через модель
with torch.no_grad():
    output = model(**encoded_input)
    logits = output.logits
    pred_class_id = logits.argmax(dim=1).item()
    pred_class_label = id_to_label[pred_class_id]

print("Исходный текст:", sample_text)
print("Предсказанный класс:", pred_class_label)


Map:   0%|          | 0/1776 [00:00<?, ? examples/s]

Map:   0%|          | 0/222 [00:00<?, ? examples/s]

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\user\AppData\Local\Temp\ipykernel_13576\1604105455.py:95: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.300700,0.082336,0.977477,0.977928,0.977477,0.977454
2,0.044200,0.150252,0.977477,0.977475,0.977477,0.977314
3,0.015100,0.126898,0.981982,0.981982,0.981982,0.981928


{'eval_loss': 0.0013865981018170714, 'eval_accuracy': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_runtime': 62.374, 'eval_samples_per_second': 3.575, 'eval_steps_per_second': 0.449, 'epoch': 3.0}
Исходный текст: A new movie starring popular actors is releasing soon.
Предсказанный класс: entertainment


In [34]:
# Явное сохранение финальной модели после обучения
trainer.save_model("distilbert_bbc_final")

business:
https://www.bbc.com/news/articles/c9djj7pgz57o
https://www.bbc.com/news/articles/cvgppr7g508o
https://www.bbc.com/news/articles/cgm88w7mjw4o
https://www.bbc.com/news/articles/cj0zz357532o
https://www.bbc.com/news/videos/cvgnnw15z83o
entertainment:
https://www.bbc.com/news/articles/c9w8g7jlkpwo
https://www.bbc.com/news/articles/cdxgkqp8g10o
https://www.bbc.com/news/articles/cm2nxxe238mo
https://www.bbc.com/news/articles/c30q87j01yyo
https://www.bbc.com/news/articles/cn80zvvy2zvo
politics:
https://www.bbc.com/news/articles/c7vnz4jy97no
https://www.bbc.com/news/articles/cdxqk1x0n5lo
https://www.bbc.com/news/articles/clyw8jw11jwo
https://www.bbc.com/news/articles/cjd3jr7e4n3o
https://www.bbc.com/news/articles/c7vnnv6n29no
sport:
https://www.bbc.com/sport/football/articles/c20xngdprpyo
https://www.bbc.com/sport/tennis/articles/cd7vg2dn39eo
https://www.bbc.com/sport/cricket/articles/cn4wgj2v2neo
https://www.bbc.com/sport/football/articles/cewggw0q4v7o
https://www.bbc.com/sport/rugby-union/articles/ckgr4dkd3pwo
tech:
https://www.bbc.com/news/articles/cj3xjrj7v78o
https://www.bbc.com/news/articles/cg4114271x2o
https://www.bbc.com/news/articles/cx28845z30go
https://www.bbc.com/future/article/20250404-where-ev-batteries-go-to-die-and-be-reborn

In [35]:
import torch
import numpy as np
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast
from torch.nn.functional import softmax
from sklearn.metrics import classification_report

# Задаем устройство (GPU, если доступно)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка сохраненной модели и токенизатора
# Загрузка модели из папки, куда она была сохранена
model_path = "distilbert_bbc_final"
model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

# Загружаем сопоставление id -> label, если оно было сохранено в модели
# Если в конфигурации модели уже задан id2label, то используем его
id_to_label = model.config.id2label if model.config.id2label is not None else None

# Перевод модели в режим инференса и на устройство
model.to(device)
model.eval()

def predict_news_bert(text, model, tokenizer, device):
    """
    Функция принимает сырой текст, токенизирует его, пропускает через модель,
    и возвращает:
       - предсказанную метку,
       - массив вероятностей (softmax) для каждого класса.
    """
    # Токенизация (с padding и truncation)
    encoded_input = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=256)
    # Переносим входные данные на device
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    with torch.no_grad():
        outputs = model(**encoded_input)
        logits = outputs.logits  # размер [1, num_labels]
        probs = softmax(logits, dim=1).cpu().numpy()[0]
    pred_idx = np.argmax(probs)
    if id_to_label is not None:
        pred_label = id_to_label[pred_idx]
    else:
        pred_label = str(pred_idx)
    return pred_label, probs

# Применяем модель к новому датасету df_new, где колонка "text" содержит новости
pred_labels = []
pred_probs_list = []

for text in df_new['text']:
    label, probs = predict_news_bert(text, model, tokenizer, device)
    pred_labels.append(label)
    pred_probs_list.append(probs)

# Добавляем результаты инференса к DataFrame
df_new['pred_label_bert'] = pred_labels
# Преобразуем вероятность в словарь {label: probability} для удобства вывода, если id_to_label задан
if id_to_label is not None:
    df_new['pred_probs_bert'] = [dict(zip(id_to_label.values(), probs)) for probs in pred_probs_list]
else:
    df_new['pred_probs_bert'] = [dict(enumerate(probs)) for probs in pred_probs_list]

# Выводим первые несколько строк с предсказаниями
print("Примеры предсказаний на новом датасете:")
print(df_new[['true_label', 'pred_label_bert', 'pred_probs_bert']].head(10))

# Если истинные метки присутствуют (столбец "true_label"), можно вычислить отчет по метрикам:
print("\nОтчет по качеству инференса на новом датасете:")
print(classification_report(df_new['true_label'], df_new['pred_label_bert']))


Примеры предсказаний на новом датасете:
      true_label pred_label_bert  \
0       business        business   
1       business        business   
2       business        business   
3       business        business   
4       business        business   
5  entertainment   entertainment   
6  entertainment   entertainment   
7  entertainment   entertainment   
8  entertainment   entertainment   
9  entertainment   entertainment   

                                     pred_probs_bert  
0  {'business': 0.99890447, 'entertainment': 0.00...  
1  {'business': 0.9988494, 'entertainment': 0.000...  
2  {'business': 0.9989189, 'entertainment': 0.000...  
3  {'business': 0.9989177, 'entertainment': 0.000...  
4  {'business': 0.99885666, 'entertainment': 0.00...  
5  {'business': 0.00023044243, 'entertainment': 0...  
6  {'business': 0.00022685429, 'entertainment': 0...  
7  {'business': 0.00020540925, 'entertainment': 0...  
8  {'business': 0.00034295965, 'entertainment': 0...  
9  {'business


## Ниже краткий сравнительный анализ результатов двух подходов:

**Классический метод (TF-IDF + Logistic Regression):**  
- Общая точность составила 76%.  
- Модели хорошо справились с классами «entertainment» и «sport» (получены идеальные показатели), однако для классов «business», «politics» и «tech» наблюдаются проблемы. Например, для «business» низкая точность (precision 0.45) при идеальной полноте (recall 1.00) говорит о большом количестве ложноположительных предсказаний, а для «politics» и «tech» f1-меры оказались ниже (~0.57).  
- Такой результат может говорить о том, что выбранный классический подход чувствителен к неидеальной предобработке и лексическим особенностям новостного текста, особенно при небольшом размере выборки (всего 5 примеров на класс).

**Fine-tuned DistilBERT:**  
- Модель показала общую точность 88% с улучшенными макро- и взвешенными метриками (macro avg f1 ≈ 0.88).  
- Особенно заметно улучшение для сложных классов: для «business» f1 вырос до 0.83, а для «politics» — до 0.75, что свидетельствует о лучшей способности модели улавливать контекстуальные признаки и нюансы языка.
- Повышенные результаты обусловлены возможностями трансформерной архитектуры извлекать глубокие семантические зависимости из текста, что особенно важно при разнообразии формулировок в новостях.

**Вывод:**  
Fine-tuned DistilBERT демонстрирует явное преимущество по качеству инференса: модель более стабильно и точно предсказывает классы новостей за счет использования контекстных эмбеддингов, чего не достигает классический алгоритм на основе TF-IDF и Logistic Regression. При этом для реальных задач, особенно когда доступно достаточное количество данных и требуются тонкие различия между категориями, модели на базе трансформеров оказываются более эффективными, несмотря на увеличение вычислительных затрат на обучение и инференс.